# 启动并行任务

异步执行可以通过线程实现，使用 {class}`~concurrent.futures.ThreadPoolExecutor` 或 {class}`~concurrent.futures.InterpreterPoolExecutor`，或者通过独立进程实现，使用 {class}`~concurrent.futures.ProcessPoolExecutor`。它们都实现了相同的接口，该接口由抽象的 {class}`~concurrent.futures.Executor` 类定义。

`concurrent.futures.Executor`
:   抽象类提供异步执行调用方法。要通过它的子类调用，而不是直接调用。


`concurrent.futures.Executor.submit(fn, /, *args, **kwargs)`
:   调度可调用对象 `fn`，以 `fn(*args, **kwargs)` 方式执行并返回代表该可调用对象的执行的 `Future` 对象。


In [1]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(pow, 323, 1235)
    print(future.result())

7330187419711662525292446729952277967765833457839424373869116780517420149076198183898894905236339668910883614705749593461204605714622607490668683222808958179218191780471063516227460897872227090014097232715359988867409006240081206335670819048146328123935337644631341482039026277834498173918554303355637321060412263715606736911839847081166018722332660742474936263046482602637679145832497919840537694829188335160914131310111239449199642739655793719812086149415859534959085359215402107080568853413877372159233452025442228651418507639010743174496936173262981681095359156359401217962539764203947129055258900852300663811552683018727645219707243611502505332407412509113706415784954450373499498470564461122438759199401776852200401606640428677779377601753155808428527723343610817938497649317098807976560094452142608380040084257453294862872175833371967399756248792099979580669182692895291737594003884626270278670234491588018888596571060169103937561057223083861024046092685665503687387204434227037124219374130642

`map(fn, *iterables, timeout=None, chunksize=1)`
:   类似于 `map(fn, *iterables)` 但有以下差异：

    `iterables` 是立即执行而不是延迟执行的；

    `fn` 是异步执行的并且可以并发对 `fn` 的多个调用。
    如果 {meth}`~concurrent.futures.Executor.__next__` 被调用且从对 {meth}`~concurrent.futures.Executor.map` 原始调用 `timeout` 秒之后其结果还不可用则已返回的迭代器将引发 {data}`TimeoutError`。 `timeout` 可以是整数或浮点数。如果 `timeout` 未指定或为 `None`，则不限制等待时间。

    如果 `fn` 调用引发了异常，那么当从迭代器获取其值时该异常将被引发。

    在使用 {class}`~concurrent.futures.ProcessPoolExecutor` 时，此方法将可迭代对象分割成若干块，然后将它们作为独立任务提交到池中。这些块的（大致）大小可以通过将 `chunksize` 设置为正整数来指定。对于非常长的可迭代对象，与默认值 `1` 相比，使用较大的 `chunksize` 可以显著提高性能。对于 {class}`~concurrent.futures.ThreadPoolExecutor` 和 {class}`~concurrent.futures.InterpreterPoolExecutor`，`chunksize` 则没有影响。

当可调用对象已关联了 {class}`~concurrent.futures.Future` 然后在等待另一个 {class}`~concurrent.futures.Future` 的结果时就会导致死锁情况。例如：
```python
import time
import concurrent.futures

def wait_on_b():
    time.sleep(5)
    print(b.result())  # b 永远不会结束因为它在等待 a。
    return 5

def wait_on_a():
    time.sleep(5)
    print(a.result())  # a 永远不会结束因为它在等待 b。
    return 6


with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    a = executor.submit(wait_on_b)
    b = executor.submit(wait_on_a)
```
与
```python
import time
import concurrent.futures


def wait_on_future():
    f = executor.submit(pow, 5, 2)
    # 这将永远不会完成因为只有一个工作线程
    # 并且它正在执行此函数。
    print(f.result())

with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    executor.submit(wait_on_future)
```

In [2]:
import concurrent.futures
import urllib.request

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://nonexistent-subdomain.python.org/']

# 检索单个网页并报告其 URL 及内容。
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

# 使用 `with` 语句来确保线程得到及时清理
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # 开始加载操作，并为每个未来任务标记其 URL
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('{%r} generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

{'http://nonexistent-subdomain.python.org/'} generated an exception: <urlopen error [Errno -2] Name or service not known>
{'http://www.foxnews.com/'} generated an exception: HTTP Error 403: Forbidden 1001
{'http://europe.wsj.com/'} generated an exception: <urlopen error [Errno 101] Network is unreachable>
{'http://www.bbc.co.uk/'} generated an exception: <urlopen error [Errno 101] Network is unreachable>
'http://www.cnn.com/' page is 3469638 bytes


## 进程池

In [3]:
import concurrent.futures
import multiprocessing as mp
import time
import math

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    print(f"{mp.current_process()} {time.asctime()}")
    return True

In [4]:
import asyncio
with concurrent.futures.ProcessPoolExecutor() as executor:
    print(f"{mp.current_process()} {time.asctime()}")
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES, chunksize=1)):
        print(f'{number:d} is prime: {prime}')
    print(f"{mp.current_process()} {time.asctime()}")
await asyncio.sleep(3) # 保证 ipykernel 单元格线程耗尽

<_MainProcess name='MainProcess' parent=None started> Sat Nov  9 15:26:43 2024
<ForkProcess name='ForkProcess-4' parent=1848926 started> Sat Nov  9 15:26:43 2024
<ForkProcess name='ForkProcess-5' parent=1848926 started> Sat Nov  9 15:26:43 2024
<ForkProcess name='ForkProcess-2' parent=1848926 started> Sat Nov  9 15:26:43 2024
<ForkProcess name='ForkProcess-1' parent=1848926 started> Sat Nov  9 15:26:43 2024
<ForkProcess name='ForkProcess-3' parent=1848926 started> Sat Nov  9 15:26:43 2024
112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
<_MainProcess name='MainProcess' parent=None started> Sat Nov  9 15:26:43 2024
